In [ ]:
%tensorflow_version 2.x 
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf


In [ ]:
CSV_COLUMN_NAMES = ['date','Summary', 'precipitation','Temperature', 'Apparent_Temperature','Humidity','Wind_Speed','Wind_Bearing','Visibility','cloud_cover','Pressure','Daily_Summary']
DAILY = ['Setosa', 'Versicolor', 'Virginica']
# Lets define some constants to help us later on

In [ ]:
train = pd.read_csv('/content/train.csv', names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv('/content/test.csv', names=CSV_COLUMN_NAMES, header=0)

In [ ]:
train[train.isnull().any(axis=1)]
train.fillna('0', inplace=True)

In [ ]:
train.isnull().sum()

date                    0
Summary                 0
precipitation           0
Temperature             0
Apparent_Temperature    0
Humidity                0
Wind_Speed              0
Wind_Bearing            0
Visibility              0
cloud_cover             0
Pressure                0
Daily_Summary           0
dtype: int64

In [ ]:
train.head()

,date,Summary,precipitation,Temperature,Apparent_Temperature,Humidity,Wind_Speed,Wind_Bearing,Visibility,cloud_cover,Pressure,Daily_Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251,15.8263,0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259,15.8263,0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204,14.9569,0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269,15.8263,0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259,15.8263,0,1016.51,Partly cloudy throughout the day.


In [ ]:
train_y = train.pop('Daily_Summary')
test_y = test.pop('Daily_Summary')
train.pop('date')
test.pop('date')
train.pop('cloud_cover')
test.pop('cloud_cover')
train.pop('Summary')
test.pop('Summary')
train.pop('precipitation')
test.pop('precipitation')

train.head() # the species column is now gone

,Temperature,Apparent_Temperature,Humidity,Wind_Speed,Wind_Bearing,Visibility,Pressure
0,9.472222,7.388889,0.89,14.1197,251,15.8263,1015.13
1,9.355556,7.227778,0.86,14.2646,259,15.8263,1015.63
2,9.377778,9.377778,0.89,3.9284,204,14.9569,1015.94
3,8.288889,5.944444,0.83,14.1036,269,15.8263,1016.41
4,8.755556,6.977778,0.83,11.0446,259,15.8263,1016.51


In [ ]:
train.shape

(94268, 7)

In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [ ]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='Temperature', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Apparent_Temperature', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Humidity', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Wind_Speed', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Wind_Bearing', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Visibility', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Pressure', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [ ]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=207)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpnsn_2gez', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),steps=5000)
# We include a lambda to avoid creating an inner function previously

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.


ValueError: ignored